# SQLAlchemy, GeoPandas and PostGIS #
This notebook covers a basic introduction to the SQLAlchemy, GeoPandas and PostGIS.

In this notebook we will...

## 0. Version Checks

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.4.44'

In [2]:
import pandas as pd
pd.__version__

'1.5.1'

In [3]:
import geopandas as gpd
gpd.__version__

'0.12.1'

## 1. Connect to the DB Cluster
GeoAlchemy relies on the `create_engine()` function from `SQLAlchemy` to manage DB connections.

In [4]:
from sqlalchemy import create_engine

In [5]:
pguser = 'matthew'
pgpassword = 'gVu*vdyt6xEPjHgc'
pghost = 'pg-testbench.internal.myheat.ca'
pgport = '5432'
pgdb = 'my_new_db'
pgconstring = f'postgresql://{pguser}:{pgpassword}@{pghost}:{pgport}/{pgdb}'

In [6]:
engine = create_engine(pgconstring, echo=True)

In [9]:
type(engine)

sqlalchemy.engine.base.Engine

In [7]:
type(engine)

sqlalchemy.engine.base.Engine

## 2. Test Our DB Connection
We can test our DB connection be loading our `polygons` table / feature class into a GeoPandas DataFrame.

To do this we will define a custom function to both handle errors and to format the GeoDataFrame:

In [8]:
def postgis2gdf(query: str, engine: sqlalchemy.engine.base.Engine) -> gpd.GeoDataFrame:
    """
    Executes an SQL query against a PostGIS database and returns
    the result as a GeoDataFrame.

    NOTE: Requires GeoPandas imported as `gpd`

    Args:
        query (str): SQL 'SELECT' query to execute 
        engine (sqlalchemy.engine.base.Engine): Engine connected to the target database

    Returns:
        gpd.GeoDataFrame: Resulting GeoDataFrame
    """
    try:
        gdf = gpd.read_postgis(query, engine, index_col='gid')
    except Exception as err:
        print(f'\n!!ERROR!! {err}')
        return None

    return gdf
    

NameError: name 'postgis2gdf' is not defined

In [ ]:
# polygons_query = "SELECT * FROM polygons WHERE name = 'Wales'"
polygons_query = "SELECT * FROM polygons"
polygons_gdf = postgis2gdf(polygons_query, engine)

In [ ]:
polygons_gdf

In [ ]:
polygons_gdf.crs

Boo-Ya! It looks like we're in business =D

## 3. Importing Spatial Data with GeoPandas ##

We can import SHP files, GeoPackages, GeoJSON, etc. all with the `read_file()` function.

However, we will wrap this in a function both to handle errors and to make sure that the resulting DataFrame conforms to our PostGIS conventions.

For this example, let's import the `points.shp` file:

In [ ]:
# Define an import function to both handle errors and to format the indices
def file2gdf(file_in: str) -> gpd.GeoDataFrame:
    """
    Reads a geometry datafile (SHP, GPKG, GeoJSON, etc)
    into a GeoDataFrame using the MyHEAT index conventions. 

    Args:
        file_in (str): Input file path

    Returns:
        gpd.GeoDataFrame: Resulting GeoDataFrame
    """
    try:
        gdf = gpd.read_file(file_in)
    except Exception as ex:
        print(ex)
        return None
    
    # Check if table already contains a `gid`
    if 'gid' in gdf.columns:
        gdf.set_index('gid')
    else:
        gdf.index.names = ['gid']
        gdf.index = gdf.index + 1
    
    gdf.rename_geometry('geom', inplace=True)

    return gdf
        

In [ ]:
fn = "./sample_data/points.shp"
points_gdf = file2gdf(fn)
points_gdf

In [ ]:
points_gdf.info()

Primarily what GeoPandas adds to regular Pandas is the `geometery` dtype.

Let's take a closer look at this:

In [ ]:
points_gdf.iloc[0]['geom'].geom_type

In [ ]:
points_gdf.iloc[0]['geom'].coords[:]

In [ ]:
points_gdf.crs

Yup, it's just that easy!

## 4. Exporting to PostGIS / Postgres ##

In this next section we will use GeoPandas to export our spatial DataFrame to our PostGIS DB.

The syntax for this is `geopandas.to_postgis('table_name', db_engine, index=True, index_label='gid')`.

Again, we will define a custom function here for error handling and to correctly setup the spatial index:

In [ ]:
# Define a function for exporting our GeoDataFrame to PostGIS following our 
def gdf2postgis(gdf: gpd.GeoDataFrame, engine: sqlalchemy.engine.base.Engine, table_name: str, if_exists='replace'):
    """
    Exports a MyHEAT GeoDataFrame to a PostGIS 

    Args:
        gdf (gpd.GeoDataFrame): Source GeoDataFrame
        engine (sqlalchemy.engine.base.Engine): DB engine connected to the destination database
        table_name (str): Name of the table to create
        if_exists: {'fail', 'replace', 'append'}, default 'replace'
    """
    try:
        gdf.to_postgis(table_name, engine, if_exists=if_exists, index=True, index_label='gid')
    except Exception as oops:
        print(f'\n!!ERROR!! {oops}')
        return None

    # Check if the sequence for gid already exists, and if not, create it
    with engine.connect() as con:
        seqs = con.execute(f'SELECT relname FROM pg_class WHERE relkind = \'S\'')
        if(f'{table_name}_gid_seq' in [i.relname for i in seqs]):
            return
        else:
            con.execute(f'CREATE SEQUENCE {table_name}_gid_seq OWNED BY {table_name}.gid;')
            con.execute(f'ALTER TABLE {table_name} ALTER COLUMN gid SET DEFAULT nextval(\'{table_name}_gid_seq\');')
            con.execute(f'ALTER TABLE {table_name} ADD PRIMARY KEY (gid);')

In [ ]:
gdf2postgis(points_gdf, engine, 'points')

## 5 Full Circle ##

Finally let's try recalling our points GeoDataFrame from our DB just to prove everything worked ;)

In [ ]:
check_points_gdf = postgis2gdf('SELECT * FROM points', engine)
check_points_gdf

BONUS: What if we only wanted the points for Berlin?

In [ ]:
berlin_points_gdf = postgis2gdf('SELECT * FROM points WHERE name = \'Berlin\'', engine)
berlin_points_gdf